In [1]:
import json
import re

In [2]:
with open("vaccination-v1.json", "r") as outfile:
    output=json.load(outfile)

In [3]:
import pandas as pd
df=pd.DataFrame(output)

In [4]:
df

,id,username,text,favourite_count,retweet_count,source,location
0,1432724865807790081,EdgarEllenPope,RT @IANPPC613: 🚨Mega thread🚨 Have you or someo...,0,5,Twitter for iPhone,
1,1432724757351440401,EastMedPulse,RT @StPapagianneas: 70% of all adults in the #...,0,6,Twitter for iPad,East Meditteranean
2,1432724719334211600,aneesinha,"1 crore doses, 2 times in 5 days\n#India 🇮🇳 ac...",0,0,Twitter for iPhone,Mumbai
3,1432724628334518275,avanindra43,RT @VikashSharmaIB: India sets another #vaccin...,0,3,Twitter for Android,गाजीपुर उत्तर प्रदेश
4,1432724388458156039,covid_nl,@CVaccinebot 🇳🇱 Vaccine rollout for the Nether...,0,0,CovidAnalyticsNL,Netherlands
...,...,...,...,...,...,...,...
1395,1432527323820003332,AusIntSociety,Getting to a #postcovid world goes through the...,2,2,Twitter Web App,"Melbourne, VIC, Australia"
1396,1432527118072692737,Q_delmo,RT @DrJohnB2: (1/n) Platelet activation in vac...,0,47,Twitter for Android,Dublin
1397,1432526990150705155,DonelAdams,RT @BNCNews: BNC's legal contributor @PaulHend...,0,5,Twitter for Android,"Youngstown, OH"
1398,1432526824588808199,TooSimple11,Gladys stuffs the management of #COVID19nsw an...,0,0,Twitter for iPad,The Canberra Bubble


In [16]:
from nltk.corpus import twitter_samples

In [24]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [5]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier

import re, string, random

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

if __name__ == "__main__":

    positive_tweets = twitter_samples.strings('positive_tweets.json')
    negative_tweets = twitter_samples.strings('negative_tweets.json')
    text = twitter_samples.strings('tweets.20150430-223406.json')
    tweet_tokens = twitter_samples.tokenized('positive_tweets.json')[0]

    stop_words = stopwords.words('english')

    positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
    negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

    positive_cleaned_tokens_list = []
    negative_cleaned_tokens_list = []

    for tokens in positive_tweet_tokens:
        positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    for tokens in negative_tweet_tokens:
        negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    all_pos_words = get_all_words(positive_cleaned_tokens_list)

    freq_dist_pos = FreqDist(all_pos_words)
    print(freq_dist_pos.most_common(10))

    positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
    negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

    positive_dataset = [(tweet_dict, "Positive")
                         for tweet_dict in positive_tokens_for_model]

    negative_dataset = [(tweet_dict, "Negative")
                         for tweet_dict in negative_tokens_for_model]

    dataset = positive_dataset + negative_dataset

    random.shuffle(dataset)

    train_data = dataset[:7000]
    test_data = dataset[7000:]

    classifier = NaiveBayesClassifier.train(train_data)

    print("Accuracy is:", classify.accuracy(classifier, test_data))

    print(classifier.show_most_informative_features(10))

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]
Accuracy is: 0.9973333333333333
Most Informative Features
                      :( = True           Negati : Positi =   2077.2 : 1.0
                      :) = True           Positi : Negati =    981.2 : 1.0
                     bam = True           Positi : Negati =     21.5 : 1.0
                 awesome = True           Positi : Negati =     20.8 : 1.0
                follower = True           Positi : Negati =     20.0 : 1.0
                     sad = True           Negati : Positi =     19.2 : 1.0
                  arrive = True           Positi : Negati =     18.4 : 1.0
                    poor = True           Negati : Positi =     17.8 : 1.0
                     x15 = True           Negati : Positi =     15.8 : 1.0
                 welcome = True           Positi : Negati =     15.7 : 1.0
None


In [10]:
analysis=[]
for i in range(0, len(df)):
    custom_tweet=df.iloc[i, df.columns.get_loc('text')]
    custom_tokens=remove_noise(word_tokenize(custom_tweet))
    analysis.append(classifier.classify(dict([token, True] for token in custom_tokens)))

In [11]:
df["Analysis"]=analysis

In [12]:
df

,id,username,text,favourite_count,retweet_count,source,location,Analysis
0,1432724865807790081,EdgarEllenPope,RT @IANPPC613: 🚨Mega thread🚨 Have you or someo...,0,5,Twitter for iPhone,,Negative
1,1432724757351440401,EastMedPulse,RT @StPapagianneas: 70% of all adults in the #...,0,6,Twitter for iPad,East Meditteranean,Positive
2,1432724719334211600,aneesinha,"1 crore doses, 2 times in 5 days\n#India 🇮🇳 ac...",0,0,Twitter for iPhone,Mumbai,Positive
3,1432724628334518275,avanindra43,RT @VikashSharmaIB: India sets another #vaccin...,0,3,Twitter for Android,गाजीपुर उत्तर प्रदेश,Positive
4,1432724388458156039,covid_nl,@CVaccinebot 🇳🇱 Vaccine rollout for the Nether...,0,0,CovidAnalyticsNL,Netherlands,Negative
...,...,...,...,...,...,...,...,...
1395,1432527323820003332,AusIntSociety,Getting to a #postcovid world goes through the...,2,2,Twitter Web App,"Melbourne, VIC, Australia",Negative
1396,1432527118072692737,Q_delmo,RT @DrJohnB2: (1/n) Platelet activation in vac...,0,47,Twitter for Android,Dublin,Negative
1397,1432526990150705155,DonelAdams,RT @BNCNews: BNC's legal contributor @PaulHend...,0,5,Twitter for Android,"Youngstown, OH",Positive
1398,1432526824588808199,TooSimple11,Gladys stuffs the management of #COVID19nsw an...,0,0,Twitter for iPad,The Canberra Bubble,Negative


In [14]:
df['Analysis'].value_counts()

Positive    875
Negative    525
Name: Analysis, dtype: int64

In [15]:
df.iloc[1399]['text']

'RT @CollectorCbe: Anbarntha Kovai Makkale,\nWe have 13,050 doses available with the 24×7 vaccination centres as on 30.08.21, 7pm. Make use o…'